In [ ]:
from pymongo import MongoClient
from bson.objectid import ObjectId

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["MCA_WT"]

# Dropping existing collections for a clean setup
db.students.drop()
db.competitions.drop()

# Sample Data for Students Collection
students_data = [
    {"student_id": "S01", "name": "Alice", "class": "FY", "participated_competitions": ["C01", "C02"]},
    {"student_id": "S02", "name": "Bob", "class": "SY", "participated_competitions": ["C01", "C03"]},
    {"student_id": "S03", "name": "Charlie", "class": "FY", "participated_competitions": ["C02"]},
    {"student_id": "S04", "name": "David", "class": "TY", "participated_competitions": ["C03", "C01"]},
    {"student_id": "S05", "name": "Eve", "class": "FY", "participated_competitions": ["C01", "C04"]},
    {"student_id": "S06", "name": "Frank", "class": "SY", "participated_competitions": ["C02", "C04"]},
    {"student_id": "S07", "name": "Grace", "class": "FY", "participated_competitions": ["C04", "C03"]},
    {"student_id": "S08", "name": "Hank", "class": "TY", "participated_competitions": ["C03"]},
    {"student_id": "S09", "name": "Ivy", "class": "FY", "participated_competitions": ["C01", "C02"]},
    {"student_id": "S10", "name": "Jack", "class": "SY", "participated_competitions": ["C02"]}
]

# Insert Students Data
db.students.insert_many(students_data)

# Sample Data for Competitions Collection
competitions_data = [
    {"competition_id": "C01", "name": "Programming", "participants": [{"student_id": "S01", "rank": 1}, {"student_id": "S02", "rank": 2}, {"student_id": "S04", "rank": 3}, {"student_id": "S05", "rank": 4}]},
    {"competition_id": "C02", "name": "E-Rangoli", "participants": [{"student_id": "S01", "rank": 1}, {"student_id": "S03", "rank": 2}, {"student_id": "S09", "rank": 3}, {"student_id": "S10", "rank": 4}]},
    {"competition_id": "C03", "name": "Debate", "participants": [{"student_id": "S02", "rank": 1}, {"student_id": "S04", "rank": 2}, {"student_id": "S07", "rank": 3}, {"student_id": "S08", "rank": 4}]},
    {"competition_id": "C04", "name": "Quiz", "participants": [{"student_id": "S05", "rank": 1}, {"student_id": "S06", "rank": 2}, {"student_id": "S07", "rank": 3}]}
]

# Insert Competitions Data
db.competitions.insert_many(competitions_data)



InsertManyResult([ObjectId('672b7f459bf5e17229558371'), ObjectId('672b7f459bf5e17229558372'), ObjectId('672b7f459bf5e17229558373'), ObjectId('672b7f459bf5e17229558374')], acknowledged=True)

In [17]:
# Queries

# a. Display average number of students participating in each competition
average_participation = db.competitions.aggregate([
    {"$project": {"name": 1, "average_participants": {"$size": "$participants"}}},
    {"$group": {"_id": None, "average_participation": {"$avg": "$average_participants"}}}
])

print("Average number of students participating in each competition:")
for result in average_participation:
    print(result)









Average number of students participating in each competition:
{'_id': None, 'average_participation': 3.75}


In [14]:

# b. Find the number of students for Programming competition
programming_participation = db.competitions.find_one({"name": "Programming"})
print(programming_participation['participants'])

cnt = len(programming_participation['participants'])

print("\nNumber of students in Programming competition:", cnt)

[{'student_id': 'S01', 'rank': 1}, {'student_id': 'S02', 'rank': 2}, {'student_id': 'S04', 'rank': 3}, {'student_id': 'S05', 'rank': 4}]

Number of students in Programming competition: 4


In [15]:



# c. Display the names of the first three winners of each competition
competitions = db.competitions.find()

print("\nFirst three winners of each competition:")
for competition in competitions:
    competition_name = competition["name"]
    
    # Directly get the first three participants (assuming they're already in rank order)
    top_three_participants = competition["participants"][:3]
    
    # Fetch student names for the top three participants
    for participant in top_three_participants:
        student = db.students.find_one({"student_id": participant["student_id"]})
        if student:
            print({
                "competition_name": competition_name,
                "student_name": student["name"],
                "rank": participant["rank"]
            })





First three winners of each competition:
{'competition_name': 'Programming', 'student_name': 'Alice', 'rank': 1}
{'competition_name': 'Programming', 'student_name': 'Bob', 'rank': 2}
{'competition_name': 'Programming', 'student_name': 'David', 'rank': 3}
{'competition_name': 'E-Rangoli', 'student_name': 'Alice', 'rank': 1}
{'competition_name': 'E-Rangoli', 'student_name': 'Charlie', 'rank': 2}
{'competition_name': 'E-Rangoli', 'student_name': 'Ivy', 'rank': 3}
{'competition_name': 'Debate', 'student_name': 'Bob', 'rank': 1}
{'competition_name': 'Debate', 'student_name': 'David', 'rank': 2}
{'competition_name': 'Debate', 'student_name': 'Grace', 'rank': 3}
{'competition_name': 'Quiz', 'student_name': 'Eve', 'rank': 1}
{'competition_name': 'Quiz', 'student_name': 'Frank', 'rank': 2}
{'competition_name': 'Quiz', 'student_name': 'Grace', 'rank': 3}


In [16]:

# d. Display students from class 'FY' who participated in 'E-Rangoli' Competition
e_rangoli_competition = db.competitions.find_one({"name": "E-Rangoli"})

print("\nFY students who participated in 'E-Rangoli' competition:")
if e_rangoli_competition:
    # Iterate over participants and filter by class 'FY'
    for participant in e_rangoli_competition["participants"]:
        student = db.students.find_one({"student_id": participant["student_id"], "class": "FY"})
        # if student found then only if condition will be True
        
        if student:
            print({"student_name": student["name"], "class": student["class"]})



FY students who participated in 'E-Rangoli' competition:
{'student_name': 'Alice', 'class': 'FY'}
{'student_name': 'Charlie', 'class': 'FY'}
{'student_name': 'Ivy', 'class': 'FY'}
